In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras


In [2]:
# Load the mnist dataset.
train_ds, test_ds = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files=True,
)


In [3]:
def preprocess_fn(data):
    image = tf.cast(data["image"], tf.float32) / 255
    label = data["label"]
    return (image, label)


train_ds = train_ds.map(preprocess_fn).batch(128).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.map(preprocess_fn).batch(128).prefetch(tf.data.AUTOTUNE)


# model definition

In [4]:
input_shape = (28, 28, 1)
num_classes = 10

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)


In [5]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.AdamW(0.001),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Set up tracking

In [6]:
import mlflow
mlflow.set_tracking_uri(uri="http://localhost:8080")

#  Method 1: MLflow Auto Logging


In [7]:
# Choose any name that you like.
mlflow.set_experiment("/mlflow-tf-keras-mnist-deeper-net-adamW")

mlflow.tensorflow.autolog()

model.fit(x=train_ds, epochs=3)


2024/07/26 19:14:57 INFO mlflow.tracking.fluent: Experiment with name '/mlflow-tf-keras-mnist-deeper-net-adamW' does not exist. Creating a new experiment.
2024/07/26 19:14:57 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.
2024/07/26 19:14:57 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of 

Epoch 1/3
464/469 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.0090 - sparse_categorical_accuracy: 0.6701

2024/07/26 19:15:05 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.


469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - loss: 1.0034 - sparse_categorical_accuracy: 0.6720
Epoch 2/3
468/469 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.2369 - sparse_categorical_accuracy: 0.9289

2024/07/26 19:15:10 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.


469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.2368 - sparse_categorical_accuracy: 0.9290
Epoch 3/3
464/469 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1755 - sparse_categorical_accuracy: 0.9479

2024/07/26 19:15:14 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.


469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.1754 - sparse_categorical_accuracy: 0.9479


2024/07/26 19:15:14 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: tuple index out of range
2024/07/26 19:15:14 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


# score 

In [8]:
score = model.evaluate(test_ds)

print(f"Test loss: {score[0]:.4f}")
print(f"Test accuracy: {score[1]: .2f}")

79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0832 - sparse_categorical_accuracy: 0.9755
Test loss: 0.0819
Test accuracy:  0.98
